In [1]:
import torchvision

# 1. 准备训练数据集
train_data = torchvision.datasets.CIFAR10(
    root='./datasets',
      train=True, 
      transform=torchvision.transforms.ToTensor(), 
      download=True
)

# 2. 准备测试数据集
test_data = torchvision.datasets.CIFAR10(
    root='./datasets',
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True
)

train_data_size = len(train_data) # 训练数据集的大小 50000张
test_data_size = len(test_data)   # 测试数据集的大小 10000张

In [2]:
from torch.utils.data import DataLoader

# 3. 利用Dataloader 加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [8]:
from torch import nn

# 4. 搭建神经网络
# 因为 CIRAR10 是一个 10 分类的数据集，所以输出的类别是 10 个
# 这个可以单独放到一个文件中，比如model.py，然后导入这个包
class TuDui(nn.Module):
    def __init__(self):
        super(TuDui, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        output = self.model(x)
        return output

In [9]:
import torch

# 插一嘴， 如何验证自己模型是否正确
# 通常是通过判断输入指定的尺寸，看看输出的尺寸是否和预期一致

if __name__=='__main__':
    tudui = TuDui()
    input = torch.ones((64, 3, 32, 32))
    output = tudui(input)
    print(output.shape)

torch.Size([64, 10])


In [5]:
# 5. 创建网络模型
tudui = TuDui()

# 6. 创建损失函数
loss_function = nn.CrossEntropyLoss()  # CrossEntropyLoss这个损失常用于分类问题

# 7. 创建优化器
learning_rate = 0.001 # 学习率， 这个通常单独作为一个变量传入优化器,  便于修改
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate, )

In [6]:
# 8. 设置训练网络的一些参数
total_train_step = 0  # 记录训练的总次数
total_test_step = 0   # 记录测试的总次数
epoch = 10            # 训练的轮数


for i in range(epoch):
    print("-----------------第 {} 轮训练开始-----------------".format(i+1))
    
    # 训练步骤开始
    for data in train_dataloader:
        img, target = data
        output = tudui(img) # 投喂到神经网络中得到一个输出
        loss = loss_function(output, target)  # 计算损失（预测输出和真实值之间的关系）

        # 开始优化（利用优化器优化模型）
        optimizer.zero_grad()  # 优化之前一定要先把梯度清零
        loss.backward()   # 利用损失函数的反向传播，得到每个参数节点的梯度
        optimizer.step()  # 对其中的参数进行一个优化

        total_train_step = total_train_step + 1

        if total_train_step % 100 == 0:
            print("训练次数：{}，Loss：{:.4f}".format(total_train_step, loss.item()))

    # 上边是完整的一轮训练流程
    # 当一轮训练结束后，如何来判断训练的结果到底怎么样，有没有达到预期，需要对训练的结果进行测试
    # 根据测试数据集上的正确率/损失，来判断模型有没有训练好
    # 测试的过程就不要调优了，需要根据现有模型来测试
    total_test_loss = 0  # 本轮测试的总损失
    with torch.no_grad():
        for data in test_dataloader:
            img, target = data
            output = tudui(img) # 将测试数据投喂到网络中
            loss = loss_function(output,target)  # 计算模型输出与真实值之间的误差
            total_test_loss = total_test_loss + loss.item()
    print("整体测试集上的Loss:{:.4f}.".format(total_test_loss))

-----------------第 1 轮训练开始-----------------
训练次数：100，Loss：2.3047
训练次数：200，Loss：2.3278
训练次数：300，Loss：2.2822
训练次数：400，Loss：2.3127
训练次数：500，Loss：2.3030
训练次数：600，Loss：2.2924
训练次数：700，Loss：2.2948
整体测试集上的Loss:360.6908.
-----------------第 2 轮训练开始-----------------
训练次数：800，Loss：2.2980
训练次数：900，Loss：2.2949
训练次数：1000，Loss：2.3059
训练次数：1100，Loss：2.3011
训练次数：1200，Loss：2.2956
训练次数：1300，Loss：2.2854
训练次数：1400，Loss：2.3018
训练次数：1500，Loss：2.2958
整体测试集上的Loss:359.3499.
-----------------第 3 轮训练开始-----------------
训练次数：1600，Loss：2.3011
训练次数：1700，Loss：2.2785
训练次数：1800，Loss：2.2982
训练次数：1900，Loss：2.2716
训练次数：2000，Loss：2.2741
训练次数：2100，Loss：2.2846
训练次数：2200，Loss：2.2664
训练次数：2300，Loss：2.2660
整体测试集上的Loss:357.4979.
-----------------第 4 轮训练开始-----------------
训练次数：2400，Loss：2.2880
训练次数：2500，Loss：2.2897
训练次数：2600，Loss：2.2698
训练次数：2700，Loss：2.2657
训练次数：2800，Loss：2.2784
训练次数：2900，Loss：2.2688
训练次数：3000，Loss：2.2572
训练次数：3100，Loss：2.2605
整体测试集上的Loss:354.1559.
-----------------第 5 轮训练开始-----------------
训练次数：3200，Loss：2.22

In [21]:
from torch.utils.tensorboard import SummaryWriter

# 这是上一个框的改进版本，添加tensorboard

# 8. 设置训练网络的一些参数
total_train_step = 0  # 记录训练的总次数
total_test_step = 0   # 记录测试的总次数
epoch = 10            # 训练的轮数


writer = SummaryWriter("./logs_model_training")

for i in range(epoch):
    print("-----------------第 {} 轮训练开始-----------------".format(i+1))
    
    # 训练步骤开始
    for data in train_dataloader:
        img, target = data
        output = tudui(img) # 投喂到神经网络中得到一个输出
        loss = loss_function(output, target)  # 计算损失（预测输出和真实值之间的关系）

        # 开始优化（利用优化器优化模型）
        optimizer.zero_grad()  # 优化之前一定要先把梯度清零
        loss.backward()   # 利用损失函数的反向传播，得到每个参数节点的梯度
        optimizer.step()  # 对其中的参数进行一个优化

        total_train_step = total_train_step + 1

        if total_train_step % 100 == 0:
            print("训练次数：{}，Loss：{:.4f}".format(total_train_step, loss.item()))
            writer.add_scalar('train_loss', loss.item(), global_step=total_train_step)

    # 上边是完整的一轮训练流程
    # 当一轮训练结束后，如何来判断训练的结果到底怎么样，有没有达到预期，需要对训练的结果进行测试
    # 根据测试数据集上的正确率/损失，来判断模型有没有训练好
    # 测试的过程就不要调优了，需要根据现有模型来测试
    total_test_loss = 0  # 本轮测试的总损失
    with torch.no_grad():
        for data in test_dataloader:
            img, target = data
            output = tudui(img) # 将测试数据投喂到网络中
            loss = loss_function(output,target)  # 计算模型输出与真实值之间的误差
            total_test_loss = total_test_loss + loss.item()
    print("整体测试集上的Loss:{:.4f}.".format(total_test_loss))

    total_test_step = total_test_step + 1
    writer.add_scalar('test_loss',total_test_loss, global_step=total_test_step)

    torch.save(tudui, "./models/tudui_{}_epoch.pth".format(i+1))
    print("模型tudui_{}_epoch.pth已保存.".format(i+1))

writer.close()

-----------------第 1 轮训练开始-----------------
训练次数：100，Loss：1.2973
训练次数：200，Loss：1.4113
训练次数：300，Loss：1.4473
训练次数：400，Loss：1.3694
训练次数：500，Loss：1.3803
训练次数：600，Loss：1.4758
训练次数：700，Loss：1.5866
整体测试集上的Loss:236.5653.
模型tudui_1_epoch.pth已保存.
-----------------第 2 轮训练开始-----------------
训练次数：800，Loss：1.2987
训练次数：900，Loss：1.3471
训练次数：1000，Loss：1.2790
训练次数：1100，Loss：1.5558
训练次数：1200，Loss：1.3196
训练次数：1300，Loss：1.2222
训练次数：1400，Loss：1.4337
训练次数：1500，Loss：1.3127
整体测试集上的Loss:234.4640.
模型tudui_2_epoch.pth已保存.
-----------------第 3 轮训练开始-----------------
训练次数：1600，Loss：1.3155
训练次数：1700，Loss：1.3248
训练次数：1800，Loss：1.6011
训练次数：1900，Loss：1.4443
训练次数：2000，Loss：1.7525
训练次数：2100，Loss：1.2563
训练次数：2200，Loss：1.1942
训练次数：2300，Loss：1.5056
整体测试集上的Loss:232.0970.
模型tudui_3_epoch.pth已保存.
-----------------第 4 轮训练开始-----------------
训练次数：2400，Loss：1.5077
训练次数：2500，Loss：1.2415
训练次数：2600，Loss：1.4628
训练次数：2700，Loss：1.4245
训练次数：2800，Loss：1.3601
训练次数：2900，Loss：1.5050
训练次数：3000，Loss：1.2911
训练次数：3100，Loss：1.4380
整体测试集上的Loss:2

In [12]:
from torch.utils.tensorboard import SummaryWriter

# 这是上一个框的改进版本，添加tensorboard，又添加了整体正确率

# 8. 设置训练网络的一些参数
total_train_step = 0  # 记录训练的总次数
total_test_step = 0   # 记录测试的总次数
epoch = 10            # 训练的轮数


writer = SummaryWriter("./logs_model_training")

for i in range(epoch):
    print("-----------------第 {} 轮训练开始-----------------".format(i+1))
    
    # 训练步骤开始
    for data in train_dataloader:
        img, target = data
        output = tudui(img) # 投喂到神经网络中得到一个输出
        loss = loss_function(output, target)  # 计算损失（预测输出和真实值之间的关系）

        # 开始优化（利用优化器优化模型）
        optimizer.zero_grad()  # 优化之前一定要先把梯度清零
        loss.backward()   # 利用损失函数的反向传播，得到每个参数节点的梯度
        optimizer.step()  # 对其中的参数进行一个优化

        total_train_step = total_train_step + 1

        if total_train_step % 100 == 0:
            print("训练次数：{}，Loss：{:.4f}".format(total_train_step, loss.item()))
            writer.add_scalar('train_loss', loss.item(), global_step=total_train_step)

    # 上边是完整的一轮训练流程
    # 当一轮训练结束后，如何来判断训练的结果到底怎么样，有没有达到预期，需要对训练的结果进行测试
    # 根据测试数据集上的正确率/损失，来判断模型有没有训练好
    # 测试的过程就不要调优了，需要根据现有模型来测试
    total_test_loss = 0  # 本轮测试的总损失
    total_accuracy = 0  # 整体的正确率
    with torch.no_grad():
        for data in test_dataloader:
            img, target = data
            output = tudui(img) # 将测试数据投喂到网络中
            loss = loss_function(output,target)  # 计算模型输出与真实值之间的误差
            total_test_loss = total_test_loss + loss.item()
            accuracy = (output.argmax(1) == target).sum()
            total_accuracy = total_accuracy + accuracy
    print("整体测试集上的Loss:{:.4f}.".format(total_test_loss))
    print("整体测试集上的准确度ACC:{:.4%}".format(total_accuracy/test_data_size))

    total_test_step = total_test_step + 1
    writer.add_scalar('test_loss',total_test_loss, global_step=total_test_step)
    writer.add_scalar('test_accuracy',total_accuracy, global_step=total_test_step)

    torch.save(tudui, "./models/tudui_{}_epoch.pth".format(i+1))
    print("模型tudui_{}_epoch.pth已保存.".format(i+1))

writer.close()

-----------------第 1 轮训练开始-----------------
训练次数：100，Loss：2.3030
训练次数：200，Loss：2.3197
训练次数：300，Loss：2.2996
训练次数：400，Loss：2.3178
训练次数：500，Loss：2.3074
训练次数：600，Loss：2.2960
训练次数：700，Loss：2.3098
整体测试集上的Loss:361.9065.
整体测试集上的准确度ACC:0.10700000077486038
模型tudui_1_epoch.pth已保存.
-----------------第 2 轮训练开始-----------------
训练次数：800，Loss：2.3038
训练次数：900，Loss：2.2942
训练次数：1000，Loss：2.3042
训练次数：1100，Loss：2.3010
训练次数：1200，Loss：2.3160
训练次数：1300，Loss：2.3045
训练次数：1400，Loss：2.3184
训练次数：1500，Loss：2.3013
整体测试集上的Loss:361.9065.
整体测试集上的准确度ACC:0.10700000077486038
模型tudui_2_epoch.pth已保存.
-----------------第 3 轮训练开始-----------------
训练次数：1600，Loss：2.3074
训练次数：1700，Loss：2.3112
训练次数：1800，Loss：2.3269
训练次数：1900，Loss：2.2939
训练次数：2000，Loss：2.3000
训练次数：2100，Loss：2.3149
训练次数：2200，Loss：2.2985
训练次数：2300，Loss：2.2905
整体测试集上的Loss:361.9065.
整体测试集上的准确度ACC:0.10700000077486038
模型tudui_3_epoch.pth已保存.
-----------------第 4 轮训练开始-----------------
训练次数：2400，Loss：2.3261
训练次数：2500，Loss：2.3088
训练次数：2600，Loss：2.2983
训练次数：2700，Loss：2.3013

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# 这是上一个框的改进版本，添加tensorboard，又添加了整体正确率， 最后优化

# 8. 设置训练网络的一些参数
total_train_step = 0  # 记录训练的总次数
total_test_step = 0   # 记录测试的总次数
epoch = 10            # 训练的轮数


writer = SummaryWriter("./logs_model_training")

for i in range(epoch):
    print("-----------------第 {} 轮训练开始-----------------".format(i+1))
    
    # 训练步骤开始
    for data in train_dataloader:
        img, target = data
        output = tudui(img) # 投喂到神经网络中得到一个输出
        loss = loss_function(output, target)  # 计算损失（预测输出和真实值之间的关系）

        # 开始优化（利用优化器优化模型）
        optimizer.zero_grad()  # 优化之前一定要先把梯度清零
        loss.backward()   # 利用损失函数的反向传播，得到每个参数节点的梯度
        optimizer.step()  # 对其中的参数进行一个优化

        total_train_step = total_train_step + 1

        if total_train_step % 100 == 0:
            print("训练次数：{}，Loss：{:.4f}".format(total_train_step, loss.item()))
            writer.add_scalar('train_loss', loss.item(), global_step=total_train_step)

    # 上边是完整的一轮训练流程
    # 当一轮训练结束后，如何来判断训练的结果到底怎么样，有没有达到预期，需要对训练的结果进行测试
    # 根据测试数据集上的正确率/损失，来判断模型有没有训练好
    # 测试的过程就不要调优了，需要根据现有模型来测试
    tudui.eval()        # 许多模型会有这一行，把网络设置为eval状态然后再进行测试
    total_test_loss = 0 # 本轮测试的总损失
    total_accuracy = 0  # 整体的正确率
    with torch.no_grad():
        for data in test_dataloader:
            img, target = data
            output = tudui(img) # 将测试数据投喂到网络中
            loss = loss_function(output,target)  # 计算模型输出与真实值之间的误差
            total_test_loss = total_test_loss + loss.item()
            accuracy = (output.argmax(1) == target).sum()
            total_accuracy = total_accuracy + accuracy
    print("整体测试集上的Loss:{:.4f}.".format(total_test_loss))
    print("整体测试集上的准确度ACC:{:.4%}".format(total_accuracy/test_data_size))

    total_test_step = total_test_step + 1
    writer.add_scalar('test_loss',total_test_loss, global_step=total_test_step)
    writer.add_scalar('test_accuracy',total_accuracy, global_step=total_test_step)

    torch.save(tudui, "./models/tudui_{}_epoch.pth".format(i+1))
    print("模型tudui_{}_epoch.pth已保存.".format(i+1))

writer.close()

In [ ]:
# 例子，比如有一个2分类问题
# input1 --> model --> [0.1, 0.2]  预测在第一个类别上的得分是0.1,在第二个类别上的得分是0,2
# input2 --> model --> [0.3, 0.4]  预测在第一个类别上的得分是0.3,在第二个类别上的得分是0.4
# 所以对模型而言，他的预测输出是 preds = [1] , [1]   # 代表两个输出都是1类别，(0就是第一个类别，1就是第二个类别)
#      假设,数据真实的标签是  targets = [0] , [1]    # 比较预测和真实：preds == targets-->[False, True].sum() = 1 ,这样就可以求出预测正确的个数
# 把输出从[0.1, 0.2] [0.3, 0.4] 转换成 [1], [1] 的格式，使用pytorch提供的Argmax()函数-->在横向上，求出最大值所在位置的下标

In [1]:
import torch

outputs = torch.tensor([
    [0.1, 0.2],
    [0.3, 0.4]
    ])

outputs.argmax(1)  # 填 1 的时候就是从横向看

tensor([1, 1])